In [1]:
!pip install boto3 astropy sfdmap progressbar2 GPUtil parse

  Using cached boto3-1.28.1-py3-none-any.whl (135 kB)
  Using cached sfdmap-0.1.1-py3-none-any.whl
  Using cached progressbar2-4.2.0-py2.py3-none-any.whl (27 kB)
  Using cached GPUtil-1.4.0-py3-none-any.whl
  Using cached parse-1.19.1-py2.py3-none-any.whl (18 kB)
  Using cached s3transfer-0.6.1-py3-none-any.whl (79 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
  Using cached botocore-1.31.1-py3-none-any.whl (11.0 MB)
  Using cached python_utils-3.7.0-py2.py3-none-any.whl (26 kB)


In [2]:
import GPUtil
print('GPUs:\n{0}'.format('\n'.join(['('+str(i+1)+')\t'+gpu.name+'\t-\t{:.2f}GB'.format(gpu.memoryFree/1e3) for i,gpu in enumerate(GPUtil.getGPUs())])))
N_GPUs = len(GPUtil.getGPUs())

# make sure tensorflow detect the GPUs
import tensorflow as tf
assert len(tf.config.list_physical_devices('GPU'))==N_GPUs, 'Not all GPUs are available!'

GPUs:
(1)	NVIDIA A100 80GB PCIe	-	nanGB


2023-07-08 17:29:08.365132: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-08 17:29:08.482725: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-08 17:29:09.165585: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-07-08 17:29:09.165674: W tensorflow/

In [3]:
# imports
import pandas as pd
import numpy as np
import os
import sys
import pickle
import boto3
from matplotlib import pyplot as plt
import time

# random seed
seed = 42
np.random.seed(seed)

# local files paths
local_home_dir_path = os.path.expanduser("~")
local_work_dir_path = os.path.join(local_home_dir_path, 'thesis2')
local_code_dir_path = os.path.join(local_work_dir_path , 'code')

# S3 file paths
endpoint_url = 'https://s3-west.nrp-nautilus.io'
bucket_name = 'tau-astro'
prefix = 'almogh'
s3_work_dir_path = os.path.join(prefix, 'thesis2')
s3_data_dir_path = os.path.join(s3_work_dir_path , 'data')
s3_models_dir_path = os.path.join(s3_work_dir_path , 'models')
s3_final_table_csv_path = os.path.join(s3_data_dir_path, 'SDSS_DR16_all.csv')

s3_client = boto3.client("s3", endpoint_url=endpoint_url)

# adding code folder to path
sys.path.insert(1, local_code_dir_path)
from s3 import to_s3_npy, to_s3_pkl, from_s3_npy, from_s3_pkl, to_s3_fig
from s3 import log_s3, s3_save_TF_model, s3_load_TF_model, from_s3_pickle
from NN import DistanceLayer, SiameseModel, DistillationDataGenerator, L1, L2

In [4]:
# Load NN
batch_size = 128
NN = s3_load_TF_model(s3_client = s3_client,
                      bucket_name = bucket_name,
                      path_in_bucket = 'almogh/thesis2/models/NN2/train/after_35_epochs/model',
                      model_name = 'model',
                      custom_objects = {'DistanceLayer': DistanceLayer, 'L1':L1, 'L2':L2, 'SiameseModel':SiameseModel})

loading from uri: s3://tau-astro/almogh/thesis2/models/NN2/train/after_35_epochs/model
INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA A100 80GB PCIe MIG 1g.10gb, compute capability 8.0


2023-07-08 17:29:38.509101: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-08 17:29:39.130386: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7925 MB memory:  -> device: 0, name: NVIDIA A100 80GB PCIe MIG 1g.10gb, pci bus id: 0000:e3:00.0, compute capability: 8.0


In [5]:
# loading outliers
data_dir = 'almogh/thesis2/data/BigRF/train'
X = from_s3_npy(s3_client = s3_client, bucket_name = bucket_name, path_in_bucket = os.path.join(data_dir, 'spec.npy'))
gs = from_s3_pickle(s3_client = s3_client, bucket_name = bucket_name, path_in_bucket = os.path.join(data_dir, 'gs.pkl'))

loading from uri: s3://tau-astro/almogh/thesis2/data/BigRF/train/spec.npy
loading from uri: s3://tau-astro/almogh/thesis2/data/BigRF/train/gs.pkl


In [6]:
# loading outliers
outliers_data_dir = 'almogh/thesis2/data/Outliers'
X_outliers = from_s3_npy(s3_client = s3_client, bucket_name = bucket_name, path_in_bucket = os.path.join(outliers_data_dir, 'spec.npy'))
gs_outliers = from_s3_pickle(s3_client = s3_client, bucket_name = bucket_name, path_in_bucket = os.path.join(outliers_data_dir, 'gs.pkl'))

loading from uri: s3://tau-astro/almogh/thesis2/data/Outliers/spec.npy
loading from uri: s3://tau-astro/almogh/thesis2/data/Outliers/gs.pkl


In [7]:
# filtering low snr
snr_th = 3
I = gs_outliers.snMedian>snr_th
print('Filtering {0}/{1} outliers with SNR lower than {2}.'.format(len(gs_outliers)-sum(I),len(gs_outliers),snr_th))
X_outliers = X_outliers[I]
gs_outliers = gs_outliers[I]

Filtering 2/114 outliers with SNR lower than 3.


In [8]:
from tensorflow.keras import utils

class DataGenerator(utils.Sequence):

    def __init__(self, x_noisy_inst, X, batch_size):

        # saving arguments
        self.x_noisy_inst = x_noisy_inst
        self.X = X
        self.batch_size = batch_size
        
        # calculating min and max
        self.x_min = x_noisy_inst.min(axis=1)
        self.x_max = x_noisy_inst.max(axis=1)
        
        # creating outliers batch (each outlier is duplicated 
        self.n_noisy_inst = x_noisy_inst.shape[0]

    def __len__(self):
        return int(np.ceil(X.shape[0]/self.batch_size))

    def __getitem__(self, index):

        y = self.X[index*self.batch_size:(index+1)*self.batch_size]
        batch_size = y.shape[0]
        
        x = np.tile(self.x_noisy_inst, (batch_size,1))
        x_min = np.tile(self.x_min, batch_size)
        x_max = np.tile(self.x_max, batch_size)
        
        # prepare a batch of the data
        y_min = y.min(axis=1)
        y_max = y.max(axis=1)
        y = np.repeat(y, self.n_noisy_inst, axis=0)
        y_min = np.repeat(y_min, self.n_noisy_inst, axis=0)
        y_max = np.repeat(y_max, self.n_noisy_inst, axis=0)
        
        # normalize
        xy_min = np.stack((x_min,y_min),axis=1).min(axis=1).reshape(-1,1)
        x -= xy_min
        y -= xy_min
        xy_max = np.stack((x_max,y_max),axis=1).max(axis=1).reshape(-1,1)
        x /= xy_max
        y /= xy_max

        return (x,y),_

In [9]:
D_NN_outliers = np.zeros(shape=(len(gs_outliers), X.shape[0]))
for i in range(len(gs_outliers)):
    print('outlier {0}/{1}'.format(i+1,len(gs_outliers)))
    x_i = X_outliers[i].reshape(1,-1)

    # infer
    data_gen = DataGenerator(x_i, X, batch_size)
    d_vec = NN.predict(data_gen)
    D_NN_outliers[i,:] = d_vec

outlier 1/112


2023-07-08 17:33:51.350383: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8500


782/782 [==============================] - 69s 83ms/step
outlier 2/112
782/782 [==============================] - 65s 83ms/step
outlier 3/112
782/782 [==============================] - 65s 83ms/step
outlier 4/112
782/782 [==============================] - 65s 83ms/step
outlier 5/112
782/782 [==============================] - 65s 83ms/step
outlier 6/112
782/782 [==============================] - 65s 83ms/step
outlier 7/112
782/782 [==============================] - 65s 83ms/step
outlier 8/112
782/782 [==============================] - 64s 82ms/step
outlier 9/112
782/782 [==============================] - 65s 83ms/step
outlier 10/112
782/782 [==============================] - 65s 83ms/step
outlier 11/112
782/782 [==============================] - 65s 83ms/step
outlier 12/112
782/782 [==============================] - 65s 83ms/step
outlier 13/112
782/782 [==============================] - 65s 83ms/step
outlier 14/112
782/782 [==============================] - 65s 83ms/step
outlier 15/112


In [11]:
# Load BigRF
BigRF = from_s3_pkl(s3_client = s3_client,
                      bucket_name = bucket_name,
                      path_in_bucket = 'almogh/thesis2/models/BigRF/train/rf.pkl')

loading from uri: s3://tau-astro/almogh/thesis2/models/BigRF/train/rf.pkl


/opt/conda/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.1.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.1.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [12]:
leafs = np.array(BigRF.apply(X), dtype=np.int16)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    0.7s finished


In [13]:
from joblib import Parallel, delayed
from uRF_SDSS import realLeaf_i
realLeafs = Parallel(n_jobs=-1, verbose=1)(delayed(realLeaf_i)(BigRF.estimators_[i].tree_.value, leafs[:, i]) for i in range(len(BigRF.estimators_)))
realLeafs = np.array(realLeafs, dtype=bool).T

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 256 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    4.4s finished


In [14]:
def dist_i_mp_outliers(i):
    
    mutLeafs = np.logical_and((leafs == outliers_leafs[i]), realLeafs, dtype=np.int16).sum(1)
    mutRealLeafs = np.logical_and(outliers_realLeafs[i], realLeafs, dtype=np.int16).sum(1, dtype=np.float64)

    return np.divide(mutLeafs, mutRealLeafs, dtype=np.float16)

In [15]:
D_BigRF_outliers = np.zeros(shape=(len(gs_outliers), X.shape[0]))
for i in range(len(gs_outliers)):
    print('outlier {0}/{1}'.format(i+1,len(gs_outliers)))
    x_i = X_outliers[i].reshape(1,-1)
    # infer
    outliers_leafs = np.array(BigRF.apply(x_i), dtype=np.int16)
    outliers_realLeafs = Parallel(n_jobs=-1, verbose=1)(delayed(realLeaf_i)(BigRF.estimators_[i].tree_.value, outliers_leafs[:, i]) for i in range(len(BigRF.estimators_)))
    outliers_realLeafs = np.array(outliers_realLeafs, dtype=bool).T
    D_BigRF_outliers[i,:] = dist_i_mp_outliers(0).reshape(1,-1)

outlier 1/112


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done 469 out of 500 | elapsed:    1.3s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    1.9s finished
/tmp/ipykernel_110/4056456762.py:3: DeprecationWarning: using `dtype=` in comparisons is only useful for `dtype=object` (and will do nothing for bool). This operation will fail in the future.
  mutLeafs = np.logical_and((leafs == outliers_leafs[i]), realLeafs, dtype=np.int16).sum(1)
/tmp/ipykernel_110/4056456762.py:4: DeprecationWarning: using `dtype=` in comparisons 

outlier 2/112


[Parallel(n_jobs=-1)]: Done 469 out of 500 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    0.6s finished
/tmp/ipykernel_110/4056456762.py:3: DeprecationWarning: using `dtype=` in comparisons is only useful for `dtype=object` (and will do nothing for bool). This operation will fail in the future.
  mutLeafs = np.logical_and((leafs == outliers_leafs[i]), realLeafs, dtype=np.int16).sum(1)
/tmp/ipykernel_110/4056456762.py:4: DeprecationWarning: using `dtype=` in comparisons is only useful for `dtype=object` (and will do nothing for bool). This operation will fail in the future.
  mutRealLeafs = np.logical_and(outliers_realLeafs[i], realLeafs, dtype=np.int16).sum(1, dtype=np.float64)
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:    0.

outlier 3/112


...

[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    0.1s finished
/tmp/ipykernel_110/4056456762.py:3: DeprecationWarning: using `dtype=` in comparisons is only useful for `dtype=object` (and will do nothing for bool). This operation will fail in the future.
  mutLeafs = np.logical_and((leafs == outliers_leafs[i]), realLeafs, dtype=np.int16).sum(1)
/tmp/ipykernel_110/4056456762.py:4: DeprecationWarning: using `dtype=` in comparisons is only useful for `dtype=object` (and will do nothing for bool). This operation will fail in the future.
  mutRealLeafs = np.logical_and(outliers_realLeafs[i], realLeafs, dtype=np.int16).sum(1, dtype=np.float64)


outlier 111/112


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    0.1s finished
/tmp/ipykernel_110/4056456762.py:3: DeprecationWarning: using `dtype=` in comparisons is only useful for `dtype=object` (and will do nothing for bool). This operation will fail in the future.
  mutLeafs = np.logical_and((leafs == outliers_leafs[i]), realLeafs, dtype=np.int16).sum(1)
/tmp/ipykernel_110/4056456762.py:4: DeprecationWarning: using `dtype=` in comparisons is only useful for `dtype=object` (and will do nothing for bool). This operation 

outlier 112/112


/tmp/ipykernel_110/4056456762.py:3: DeprecationWarning: using `dtype=` in comparisons is only useful for `dtype=object` (and will do nothing for bool). This operation will fail in the future.
  mutLeafs = np.logical_and((leafs == outliers_leafs[i]), realLeafs, dtype=np.int16).sum(1)
/tmp/ipykernel_110/4056456762.py:4: DeprecationWarning: using `dtype=` in comparisons is only useful for `dtype=object` (and will do nothing for bool). This operation will fail in the future.
  mutRealLeafs = np.logical_and(outliers_realLeafs[i], realLeafs, dtype=np.int16).sum(1, dtype=np.float64)


In [24]:
NN_scores = np.mean(D_NN_outliers,axis=1)
BigRF_scores = np.mean(D_BigRF_outliers,axis=1)

In [27]:
gs_outliers['NN_score'] = NN_scores
gs_outliers['BigRF_score'] = BigRF_scores

In [33]:
gs_outliers.sort_values(by='NN_score',ascending=False).iloc[0]

specobjid           2255301266211104768
snMedian                       13.32238
run2d                                26
ra                            186.15299
dec                           37.410154
plate                              2003
mjd                               53442
fiberid                             450
waveMin                        3811.536
waveMax                        9215.099
z                               0.04038
key                      2003_53442_450
weirdness                          0.69
spectrotype                      GALAXY
subclass                      STARBURST
oii_3726_eqw                  -54.74792
oii_3729_eqw                  -54.74792
neiii_3869_eqw                -26.28569
h_delta_eqw                   -18.99245
h_gamma_eqw                   -37.46724
oiii_4363_eqw                 -9.999014
h_beta_eqw                    -104.2134
oiii_4959_eqw                 -188.1588
oiii_5007_eqw                 -583.3107
hei_5876_eqw                  -16.69367


In [34]:
to_s3_pkl(gs_outliers, s3_client, bucket_name, 'almogh/thesis2/eval/gs_outliers_with_scores.pkl')

saving to uri: s3://tau-astro/almogh/thesis2/eval/gs_outliers_with_scores.pkl


True